# Running a Power Flow

We load the simple example network from the create_network tutorial from the pandapower.networks module:

In [ ]:
from pandapower.networks import example_simple
from pandapower.run import runpp, set_user_pf_options

In [ ]:
net = example_simple()
net

## Run a Power Flow and Access Results

Runing a loadflow adds seperate result table with the prefix 'res_':

In [ ]:
runpp(net)

In [ ]:
net

These results tables are pandas datafarmes with the same index as the element table. For example, the bus table contains all bus voltages and summed bus power injections:

In [ ]:
net.res_bus

We can now use pandas functionality to analyse the loadflow results, for example to get the minimum voltage in the medium voltage level:

In [ ]:
net.res_bus[net.bus.vn_kv==20.].vm_pu.min()

or the maxium voltage at a bus with load or generation:

In [ ]:
load_or_generation_buses = set(net.load.bus.values) | set(net.sgen.bus.values) | set(net.gen.bus.values)
net.res_bus.vm_pu.loc[list(load_or_generation_buses)].max()

For more on how to use pandas for data analysis in pandapower, see the tutorial on [data analysis](data_analysis.ipynb).

## Result tables

Each element (except the switch) has its own result table with results tailored to the specific element. Here, we just show each table. For parameters definitions, see the documentation of the datastructure.

In [ ]:
net.res_bus

In [ ]:
net.res_ext_grid

In [ ]:
net.res_line

In [ ]:
net.res_trafo

In [ ]:
net.res_load

In [ ]:
net.res_sgen

In [ ]:
net.res_gen

In [ ]:
net.res_shunt

## Voltage Angles and Initialization

Maybe you wondered why even though there is a voltage angle of 50 degrees defined for the external grid: 

In [ ]:
net.ext_grid.va_degree

and a shift of 150° over the HV/MV transformer:

In [ ]:
net.trafo.shift_degree

the voltage angles are all close to zero:

In [ ]:
runpp(net)
net.res_bus.va_degree

That is because the standard parameter for calculate_voltage_angles is False, which means voltage angles at external grids and transformer shifts are ignored by default. In a radial network, the absolute voltage angle shifts do not have an influence on the power flow, which is why they are disabled by default. In meshed networks however, where multiple external grids are galvanically coupled, it is always necessary to calculate the voltage angles.

Suppose we want to calculate the correct voltage angles and set calculate_voltage_angles to True:

In [ ]:
runpp(net, calculate_voltage_angles=True)

Now the power flow does not converge. This can happen with large angle shifts. The solution is to use a initialization with a DC loadflow instead of a flat start, which is default behaviour:

In [ ]:
runpp(net, calculate_voltage_angles=True, init="dc")

Now, we can see that all voltage angles are correctly calculated:

In [ ]:
net.res_bus.va_degree

If we already have a solution, we can also initialize the loadflow with the voltage values from the last loadflow:

In [ ]:
runpp(net, calculate_voltage_angles=True, init="results")
net.res_bus.va_degree

The power flow converges and yields correct results where a flat start power flow would have failed.

Initializing with previous results can save convergence time in cases where multiple power flows with simliar input parameters are carried out consecutively, such as in quasi-static time series simulations.

## Transformer Model

The parameter "trafo_model" can be used to switch between a 'pi' and a 't' transformer model:

In [ ]:
runpp(net, trafo_model="t")
net.res_trafo

In [ ]:
runpp(net, trafo_model="pi")
net.res_trafo

For a definition of the different transformer model see the power flow model documentation of the transformer element.

## Transformer Loading

The transformer loading can either be calculated in relation to the rated current:

In [ ]:
runpp(net, trafo_loading="current")
net.res_trafo

or to the rated power of the transformer:

In [ ]:
runpp(net, trafo_loading="power")
net.res_trafo

The transformer loading does not have an influence on other power flow results besides the loading_percent parameter.

## Generator Reactive Power Limits

The generator has reactive power limits of -3/+3 Mvar:

In [ ]:
net.gen

which are however exceeded in the power flow results, because the enforce_q_lims option defaults to False:

In [ ]:
runpp(net)
net.res_gen

If the enforce_q_lims parameter is set to True, the reactive power limit is complied with, while the voltage deviates from the voltage set point of the generator:

In [ ]:
runpp(net, enforce_q_lims=True)
net.res_gen

If you want to know what to do when a power flow does not converge, continue with the [diagnostic tutorial](diagnostic.ipynb).

## Changing the Power Flow Algorithm

There are 5 algorithms available for solving the power flow problem:
* "nr" **Newton-Raphson** - default algorithm option
* "bfsw" **Backward/Forward Sweep** (specially suited for radial and weakly-meshed networks)
* "gs" **Gauss-Seidel** (pypower implementation)
* "fdbx" **Fast-Decoupled**  power flow using XB method (pypower implementation)
* "fdxb" **Fast-Decoupled**  power flow using BX method (pypower implementation)

Each algorithm can be selected by passing corresponding string {"nr", "bfsw", "gs", "fdbx", "fdxb"} to the parameter algorithm.  
For example, if you want to use the **Backward/Forward sweep** algorithm:

In [ ]:
runpp(net, algorithm="bfsw")

Or in the case of **Gauss-Seidel**:

In [ ]:
runpp(net, algorithm="gs")

If power flow is run without setting the algorithm parameter, **Newton-Raphson** will be used as the default algorithm option.

In [ ]:
runpp(net)

There is also possibility to select **maximum number of iterations** that will be used for the specific algorithm.  
In the following example max_iteration is set to 10, which is obviously not enough for the Gauss-Seidel to converge:

In [ ]:
runpp(net, algorithm="gs", max_iteration=10)

## Setting User Options

It is possible to set user options that override the pandapower default parameters for one specific network. For the example network, the voltage angles are not calculated by default:

In [ ]:
runpp(net)
net.res_bus

We now set the option calculate_voltage_angles to True with the set_user_pf_options function:

In [ ]:
set_user_pf_options(net, calculate_voltage_angles=True, init="dc")

If we run another power flow without specifing parameters, the voltage angles are calculated:

In [ ]:
runpp(net)
net.res_bus

This change in standard behaviour is only valid for this one network.

When a parameter is specified directly in the runpp function, it overrides the user option:

In [ ]:
runpp(net, calculate_voltage_angles=False)
net.res_bus

The hierarchy for power flow options is therefore:
    1. Arguments passed to runpp
    2. User Options
    3. runpp default parameters
Note however that there is a small exception for this rule that you have to deal with: When setting your own user options and then trying to override them with the same value as the runpp default parameter, this will not be recongnized within the powerflow and the user option value is used.